In [534]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(100)
from sklearn.model_selection import train_test_split

In [363]:
class neira:
    def __init__(self, inputn=3, hiddenn=3, outputn=3, lrate=0.3):
        self.inn = inputn
        self.hn = hiddenn
        self.on = outputn
        self.lr = lrate
        self.act_func = lambda x: scipy.special.expit(x)
        self.wih = np.random.normal(0.0, pow(self.hn,-0.5), (self.hn, self.inn))
        self.who = np.random.normal(0.0, pow(self.on, -0.5), (self.on, self.hn))
        
    def train(self,inputs_list, tagets_list):
        inputs = np.array(inputs_list, ndmin = 2).T
        tagets = np.array(tagets_list, ndmin = 2).T

        hd_inn = np.dot(self.wih, inputs)

        hd_out = self.act_func(hd_inn)

        fin_inn = np.dot(self.who, hd_out)

        fin_out = self.act_func(fin_inn)

        out_err = tagets - fin_out
        hd_err = np.dot(self.who.T, out_err)

        a= self.lr* np.dot((out_err*fin_out*(1-fin_out)), np.transpose(hd_out))

        self.who += self.lr* np.dot((out_err*fin_out*(1-fin_out)), np.transpose(hd_out))
        self.wih += self.lr* np.dot((hd_err*hd_out*(1-hd_out)), np.transpose(inputs))

    def query(self,inputs_list):
        inputs = np.array(inputs_list, ndmin = 2).T
        hd_inn = np.dot(self.wih, inputs)
        
        hd_out = self.act_func(hd_inn)
        fin_inn = np.dot(self.who, hd_out)
        fin_out = self.act_func(fin_inn)
        return fin_out
        

In [630]:
data_pd = pd.read_csv('train.csv', index_col = 'Id')

for column in data_pd.columns:
    data_pd[column]= (data_pd[column]-data_pd[column].min())/(data_pd[column].max()-data_pd[column].min())

data_X = data_pd.iloc[:,:-1]
data_X
data_y = data_pd.iloc[:,-1]


In [627]:
from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(data_X, 
                                                                                     data_y, 
                                                                                     test_size = 0.3,
                                                                                     random_state =100)


In [615]:
input_nodes = 11
hidden_nodes = 8
output_nodes = 2

In [616]:
n=neira(input_nodes, hidden_nodes, output_nodes, lrate = 0.03)

In [620]:
epoch = 50
for e in range(epoch):
    for i in range(len(data_X)):
        inputs =  data_X.iloc[i,:]
        targets = np.zeros(output_nodes)+0.01       
        targets[int(data_y.iloc[i])] = 0.99
        n.train(inputs, targets)

In [628]:
pred = []
scorecard = []

for i in range(len(test_data)):


    inputs =  test_data.iloc[i,:]
    outputs = n.query(inputs)
    pred.append(outputs[1])


In [622]:
roc_auc_score(test_labels,pred)

0.8686534866025036

In [623]:
test_data1 = pd.read_csv('test.csv', index_col = 'Id')

In [624]:
for column in test_data1.columns:
    test_data1[column]= (test_data1[column]-test_data1[column].min())/(test_data1[column].max()-test_data1[column].min())


In [629]:
pred = []
scorecard = []

for i in range(len(test_data1)):

    inputs =  test_data1.iloc[i,:]
    outputs = n.query(inputs)
    pred.append(*outputs[1])

In [626]:

pd_pred= pd.DataFrame({'Id':data_test.index, 'choose': pred}, )
pd_pred.to_csv("submission.csv", index=False)